# Classify yamnet embeddings with sklearn methods

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import glob
from pathlib import Path
from tqdm import tqdm

## Get only active seconds from true positives and corresponding embeddings

## Training data

In [2]:
# init lists
X, y = [], []

# get all true positive csvs
csvs = sorted(glob.glob('./rfcx-species-audio-detection/train_tp/*.csv'))

# iterate through csvs
for csv in tqdm(csvs):
    
    # get single csvs
    gt = pd.read_csv(csv, index_col=0).to_numpy()
    
    # get frames and classes where true positives are
    specs, frames = gt.nonzero()

    # load feature embeddings
    emb = np.load(f'embeddings/yamnet/train/{Path(csv).stem}.npy')
    
    # iterate through active tp frames
    for spec, frame in zip(specs, frames):
        X.append(emb[frame, :])
        y.append(gt[:, frame])


# convert to numpy array
X = np.array(X)
y = np.array(y)

# convert one-hot to argmax
y_one_hot = y
y = np.argmax(y, axis=1)

print(X.shape, y.shape)

100%|██████████| 1132/1132 [00:08<00:00, 134.32it/s]


(4122, 1024) (4122,)


## Test data

In [18]:
# init lists
X_test = []

# get all true positive csvs
files = sorted(glob.glob('./embeddings/yamnet/test/*.npy'))

# iterate through csvs
for file in tqdm(files):
    
    # load feature embeddings
    emb = np.load(file)
    X_test.append(emb)
    
# convert to numpy array
X_test = np.array(X_test)

print(X_test.shape)

100%|██████████| 1992/1992 [00:03<00:00, 645.61it/s]


(1992, 60, 1024)


### Make tensorflow dataset from embeddings

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((X, y_one_hot))
print(dataset)

<TensorSliceDataset shapes: ((1024,), (24,)), types: (tf.float32, tf.float64)>


### Make X, y dataset

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=11)
X_train, X_val, y_one_hot_train, y_one_hot_val = train_test_split(X, y_one_hot, random_state=11)

### LRAP Metric

In [24]:
from sklearn.metrics import classification_report
from sklearn.metrics import label_ranking_average_precision_score

def get_lrap(X_val, y_one_hot_val, clf):
    y_prob = clf.predict_proba(X_val)
    y_pred = clf.predict(X_val)

    print('LRAP: %s' % label_ranking_average_precision_score(y_one_hot_val, y_prob))
    print(classification_report(y_val, y_pred))
    
def get_lrap_keras(X_val, y_one_hot_val, clf):
    y_prob = clf.predict(X_val)
    y_pred = np.argmax(y_prob, axis=1)

    print('LRAP: %s' % label_ranking_average_precision_score(y_one_hot_val, y_prob))
    print(classification_report(y_val, y_pred))

### Classify with Gradient Boosting

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

clf_xgb = GradientBoostingClassifier(random_state=0, verbose=1)
clf_xgb.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           2.7210            7.43m
         2           2.5605            7.37m
         3           2.4388            7.31m
         4           2.3415            7.22m
         5           2.2546            7.13m
         6           2.1782            7.06m
         7           2.1110            7.02m
         8           2.0373            6.97m
         9           1.9720            6.99m
        10           1.9088            7.00m
        20           1.4930            6.24m
        30           1.2263            5.52m
        40           1.0332            4.71m
        50           0.8838            3.96m
        60           0.7669            3.17m
        70           0.6713            2.40m
        80           0.5902            1.60m
        90           0.5221           48.09s
       100           0.4638            0.00s


GradientBoostingClassifier(random_state=0, verbose=1)

In [11]:
# get metrics for gradient boosting
get_lrap(X_val, y_one_hot_val, clf_xgb)

LRAP: 0.44245763985637226
              precision    recall  f1-score   support

           0       0.18      0.12      0.15        16
           1       0.18      0.08      0.11        25
           2       0.33      0.27      0.30        26
           3       0.00      0.00      0.00        20
           4       0.32      0.27      0.29        41
           5       0.23      0.05      0.09        55
           6       0.22      0.10      0.14        39
           7       0.25      0.11      0.15        56
           8       0.28      0.49      0.36        49
           9       0.20      0.13      0.16        38
          10       0.21      0.10      0.13        42
          11       0.07      0.04      0.05        26
          12       0.18      0.14      0.16        42
          13       0.35      0.35      0.35        17
          14       0.15      0.09      0.11        23
          15       0.37      0.35      0.36        48
          16       0.21      0.15      0.17        27
 

### Classify with Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier

clf_tree = DecisionTreeClassifier(random_state=8)
clf_tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=8)

In [13]:
# get metrics for gradient boosting
get_lrap(X_val, y_one_hot_val, clf_tree)

LRAP: 0.20324118978338013
              precision    recall  f1-score   support

           0       0.11      0.12      0.11        16
           1       0.00      0.00      0.00        25
           2       0.00      0.00      0.00        26
           3       0.19      0.15      0.17        20
           4       0.29      0.17      0.22        41
           5       0.15      0.09      0.11        55
           6       0.12      0.13      0.12        39
           7       0.09      0.07      0.08        56
           8       0.25      0.33      0.28        49
           9       0.15      0.18      0.16        38
          10       0.13      0.14      0.13        42
          11       0.09      0.08      0.08        26
          12       0.09      0.10      0.09        42
          13       0.16      0.18      0.17        17
          14       0.09      0.04      0.06        23
          15       0.15      0.12      0.14        48
          16       0.05      0.07      0.06        27
 

### Classify with SVM

In [16]:
from sklearn.svm import SVC

clf_svc = SVC(gamma='auto', probability=True, verbose=1)
clf_svc.fit(X, y)

[LibSVM]

SVC(gamma='auto', probability=True, verbose=1)

In [17]:
get_lrap(X_val, y_one_hot_val, clf_svc)

LRAP: 0.47440168500060803
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        16
           1       1.00      0.08      0.15        25
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00        20
           4       0.51      0.44      0.47        41
           5       1.00      0.13      0.23        55
           6       0.00      0.00      0.00        39
           7       0.00      0.00      0.00        56
           8       0.37      0.63      0.47        49
           9       1.00      0.05      0.10        38
          10       0.43      0.21      0.29        42
          11       0.00      0.00      0.00        26
          12       0.82      0.33      0.47        42
          13       0.00      0.00      0.00        17
          14       1.00      0.17      0.30        23
          15       1.00      0.10      0.19        48
          16       1.00      0.19      0.31        27
 

/Users/fabianseipel/opt/miniconda3/envs/rainforest/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Classify with Vanilla Neural Net

In [52]:
# built large model
vnn = tf.keras.Sequential([
      tf.keras.Input(shape=(1024,)),
      tf.keras.layers.Dense(512, activation='elu'),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(24, activation='softmax')
      ], name="vnn")

# optimizer
opt = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, clipnorm=1.0)
opt = tf.keras.optimizers.Nadam(lr=0.001)

# compile model
vnn.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
# train model
vnn.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(X_val, y_val))

Epoch 1/100
97/97 [==============================] - 1s 7ms/step - loss: 5.3845 - accuracy: 0.1660 - val_loss: 3.5790 - val_accuracy: 0.2037
Epoch 2/100
97/97 [==============================] - 1s 6ms/step - loss: 3.0430 - accuracy: 0.2155 - val_loss: 2.6400 - val_accuracy: 0.2493
Epoch 3/100
97/97 [==============================] - 1s 6ms/step - loss: 2.7810 - accuracy: 0.2268 - val_loss: 2.5402 - val_accuracy: 0.2580
Epoch 4/100
97/97 [==============================] - 1s 6ms/step - loss: 2.7142 - accuracy: 0.2478 - val_loss: 2.5247 - val_accuracy: 0.2502
Epoch 5/100
97/97 [==============================] - 1s 7ms/step - loss: 2.7017 - accuracy: 0.2478 - val_loss: 2.5641 - val_accuracy: 0.2609
Epoch 6/100
97/97 [==============================] - 1s 6ms/step - loss: 2.5686 - accuracy: 0.2698 - val_loss: 2.5050 - val_accuracy: 0.2755
Epoch 7/100
97/97 [==============================] - 1s 6ms/step - loss: 2.5748 - accuracy: 0.2695 - val_loss: 2.4291 - val_accuracy: 0.2881
Epoch 8/100
9

In [53]:
get_lrap_keras(X_val, y_one_hot_val, vnn)

LRAP: 0.5029935654623648
              precision    recall  f1-score   support

           0       0.15      0.19      0.17        16
           1       0.20      0.08      0.11        25
           2       0.52      0.42      0.47        26
           3       0.00      0.00      0.00        20
           4       0.31      0.41      0.35        41
           5       0.35      0.11      0.17        55
           6       0.44      0.36      0.39        39
           7       0.27      0.07      0.11        56
           8       0.38      0.55      0.45        49
           9       0.16      0.24      0.19        38
          10       0.28      0.12      0.17        42
          11       0.10      0.08      0.09        26
          12       0.43      0.48      0.45        42
          13       0.43      0.53      0.47        17
          14       0.20      0.04      0.07        23
          15       0.38      0.38      0.38        48
          16       0.36      0.33      0.35        27
  

## Predict test data